In [33]:
import numpy as np
import pickle

import matplotlib.pyplot as plt
import os
import sys
from scipy.interpolate import griddata
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colorbar
from matplotlib import rc
mpl.rcParams['axes.linewidth'] = 3
import matplotlib as mpl  
mpl.rc('font',family='Times New Roman')
from matplotlib.pyplot import cm
from scipy.interpolate import CubicSpline

from scipy.optimize import fsolve

In [72]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [34]:
os.getcwd()

'/Users/roy.369/Documents/Thermal_transport_Hubbard_model/Nearest_neighbor_Hubbard/FHM_Legacy_data/FHM_Roy_data/Text_files'

In [35]:
def fermi_distribution(en,beta):
    return 1.0/(np.exp(en*beta)+1.0)

In [36]:
def r_point_upper_half_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_1 = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              r_lab.append(r_pair_1)  
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_1 = r_pair_1+1

    R_relative_1 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_1,r_pair_1


def r_point_full_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_2 = 0
    for i in range(x_span):
        for j in range(x_span):
              r_lab.append(r_pair_2)
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_2 = r_pair_2+1

    R_relative_2 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_2,r_pair_2

In [37]:
def plot_func_multiple_beta(N,u,mu,dtau,trot,plot_title,plot_x_label,plot_y_label,x_val,y_val,y_val_var,plot_name):

   color_1 = iter(cm.gnuplot(np.linspace(0, 1, len(trot)+1)))
   plt.figure(figsize = (25,20))
   #plt.title(plot_title,fontsize = 60)
   plt.xticks(fontsize = 80)
   plt.yticks(fontsize = 80)
   #plt.ylabel(plot_y_label,fontsize = 60)
   #plt.xlabel(plot_x_label,fontsize = 60)
   for jj in range(len(trot)-1,-1,-1):
       c_1 = next(color_1)   
       beta = (float(dtau))*(float(trot[jj]))
       T = round(1/beta,2)
       plt.errorbar(x_val[:,jj],y_val[:,jj],yerr = np.sqrt(y_val_var[:,jj]),c=c_1,marker = 'o',markersize = 15, linewidth = 5, elinewidth = 3, capsize = 5,label = r"$T = %s$"%str(T))
   plt.grid()
   #plt.xlim(1.1,1.3)
   plt.axvline(x = 1,color="black", linestyle=":")
   plt.axhline(y = 0,color="black", linestyle=":")
   plt.legend(fontsize = 40, loc = 'upper right')
   plt.savefig(plot_name)
   plt.close()

In [38]:
def number_density_func(mu_val,num_den,beta):

   y = 0

   dx = np.pi/50
   dy = np.pi/50

   x_grid_cord = np.arange(0,np.pi,dx)
   y_grid_cord = np.arange(0,np.pi,dy)

   Ek = 0 #np.zeros((len(x_grid_cord),len(y_grid_cord)))
   nk_noninteracting = 0 #np.zeros((len(x_grid_cord),len(y_grid_cord)))

   t = 1
   bz_area = 0

   for i in range(len(x_grid_cord)):
       for j in range(len(y_grid_cord)):
           Ek = -2*t*(np.cos(x_grid_cord[i])+np.cos(y_grid_cord[j]))-mu_val
           nk_noninteracting = nk_noninteracting+2*fermi_distribution(Ek,beta)
           bz_area = bz_area+1
   
   Nk = nk_noninteracting/bz_area
   y = Nk-num_den

   return y

In [39]:
def adj_mat_calc(N):

    r_grid, r_rel_grid_size = r_point_upper_half_grid(int(N))
    r_rel_x = r_grid[:,0]
    r_rel_y = r_grid[:,1]

    r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
    r_2_unq = np.sort(np.unique(r_2))

    rx_cord = []
    ry_cord = []
    r2 = []
    rx = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    ry = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    for i in range(len(rx)):
        for j in range(len(ry)):
            rx_cord.append(rx[i])
            ry_cord.append(ry[j])
            r2.append(rx[i]*rx[i]+ry[j]*ry[j])

    #plt.figure()
    #plt.scatter(rx_cord,ry_cord,c=r2)

    adj_mat = np.zeros(len(r_2_unq))
    for i in range(len(r_2_unq)):
        count = 0
        for i2 in range(len(r2)):
            if round(r2[i2],2) == round(r_2_unq[i],2):
               count = count+1
        adj_mat[i] = count
    
    return np.stack((r_2_unq,np.asarray(adj_mat)),axis = 1)  

In [40]:
def non_intercting_moment_moment_corr(density,beta):

    mu_ni = fsolve(number_density_func,[0.0],args = (density,beta))

    Kx = np.arange(0,np.pi,np.pi/50)
    Ky = np.arange(0,np.pi,np.pi/50)

    bz_vol = 0
    chi_k = 0
    for i in range(len(Kx)):
        for j in range(len(Ky)):
            ek = -2*(np.cos(Kx[i])+np.cos(Ky[j]))-mu_ni[0]
            gamk = 2*(np.cos(Kx[i])+np.cos(Ky[j]))
            chi_k = chi_k+gamk*fermi_distribution(ek,beta)
            bz_vol = bz_vol+1
    
    Chi_k = chi_k/bz_vol
    m2_m2_corr = 2*density*density*Chi_k-2*Chi_k*Chi_k+4*(Chi_k**4)-2*((density*Chi_k)**2)
    
    return m2_m2_corr
    

In [41]:
def equal_time_spin_spin_connected_correlation_function_real_space_plot(Main_text_dir,Graph_dir_spin_spin,N,u,Mu,dtau,L):
   
   Adj_list = adj_mat_calc(N)
   #print("Adj_list",Adj_list)
   Density = np.zeros((len(Mu),len(L)))
    
   Up_occupancy = np.zeros((len(Mu),len(L)))
   Dn_occupancy = np.zeros((len(Mu),len(L)))
   Nup_Ndn = np.zeros((len(Mu),len(L)))
 
   Density_dev = np.zeros((len(Mu),len(L)))
   Up_occupancy_dev = np.zeros((len(Mu),len(L)))
   Dn_occupancy_dev = np.zeros((len(Mu),len(L)))
   Nup_Ndn_dev = np.zeros((len(Mu),len(L)))

   r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
   r_rel_x = r_grid[:,0]
   r_rel_y = r_grid[:,1]


   r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))

   r_2_unq = np.sort(np.unique(r_2))
   
   Spin_xx_correlation_connected = np.zeros((len(Mu),len(L),len(r_2_unq)))
   Spin_xx_correlation_connected_var = np.zeros((len(Mu),len(L),len(r_2_unq)))
   Spin_xx_correlation_connected_tot = np.zeros((len(Mu),len(L)))
   Spin_xx_correlation_connected_tot_var = np.zeros((len(Mu),len(L)))    

   Spin_zz_correlation_connected = np.zeros((len(Mu),len(L),len(r_2_unq)))
   Spin_zz_correlation_connected_var = np.zeros((len(Mu),len(L),len(r_2_unq)))
   Spin_zz_correlation_connected_tot = np.zeros((len(Mu),len(L)))
   Spin_zz_correlation_connected_tot_var = np.zeros((len(Mu),len(L)))  

   Mag_sus_xx = np.zeros((len(Mu),len(L)))
   Mag_sus_xx_std = np.zeros((len(Mu),len(L)))
   Mag_sus_zz = np.zeros((len(Mu),len(L)))
   Mag_sus_zz_std = np.zeros((len(Mu),len(L)))
    
   mu_val = np.zeros((len(Mu),len(L)))

   for i in range(len(Mu)):
       for j in range(len(L)):
        mu_val[i][j] = float(Mu[i])


   Text_dir_mag_sus = "%s/Text_files_N_%s_magnetic_susceptibility/Text_files_N_%s_U_%s"%(Main_text_dir,N,N,u)
   if not os.path.exists(Text_dir_mag_sus):
      os.makedirs(Text_dir_mag_sus)
       
   for j in range(len(L)):

       for i in range(len(Mu)):

              Text_dir_eqm = '%s/Text_files_N_%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements'%(Main_text_dir,N,N,u,dtau,Mu[i],dtau,L[j])
              Text_dir_eq_spin_conn_r = '%s/Text_files_N_%s_real_space_correlations/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Spin_spin_connected_correlation_functions'%(Main_text_dir,N,N,u,dtau,Mu[i],dtau,L[j])

              filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u,Mu[i],dtau,L[j])
              with open(filename_eqm_avg, 'rb') as infile:
                   sys_measure_avg = pickle.load(infile)

              filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u,Mu[i],dtau,L[j])
              with open(filename_eqm_std, 'rb') as infile:
                   sys_measure_std = pickle.load(infile)


              Density[i][j] = sys_measure_avg['Density averaged']
              Up_occupancy[i][j] = sys_measure_avg['Up spin occupancy averaged']
              Dn_occupancy[i][j] = sys_measure_avg['Down spin occupancy averaged']
              Nup_Ndn[i][j] = sys_measure_avg['Doublon number averaged']

              Density_dev[i][j] = sys_measure_std['Density standard deviation']
              Up_occupancy_dev[i][j] = sys_measure_std['Up spin occupancy standard deviation']
              Dn_occupancy_dev[i][j] = sys_measure_std['Down spin occupancy standard deviation']
              Nup_Ndn_dev[i][j] = sys_measure_std['Doublon number standard deviation']
           

              filename_eq_spin_xx_conn_r_nbr_avg = '%s/Equal_time_SxSx_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat' %(Text_dir_eq_spin_conn_r,N,u,Mu[i],dtau,L[j])
              r_sep_2,eq_sxsx_conn_r_nbr_avg,eq_sxsx_conn_r_nbr_std_avg = np.loadtxt(filename_eq_spin_xx_conn_r_nbr_avg,unpack = 'True',usecols = [0,1,2])
              filename_eq_spin_zz_conn_r_nbr_avg = '%s/Equal_time_SzSz_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat' %(Text_dir_eq_spin_conn_r,N,u,Mu[i],dtau,L[j])
              r_sep_2,eq_szsz_conn_r_nbr_avg,eq_szsz_conn_r_nbr_std_avg = np.loadtxt(filename_eq_spin_zz_conn_r_nbr_avg,unpack = 'True',usecols = [0,1,2])

              
              Spin_xx_correlation_connected[i,j,:] = np.copy(eq_sxsx_conn_r_nbr_avg)
              Spin_xx_correlation_connected_var[i,j,:] = np.power(eq_sxsx_conn_r_nbr_std_avg,2)                       
              Spin_xx_correlation_connected_tot[i,j] = np.dot(eq_sxsx_conn_r_nbr_avg,Adj_list[:,1])
              Spin_xx_correlation_connected_tot_var[i,j] = np.dot(np.power(eq_sxsx_conn_r_nbr_std_avg,2),Adj_list[:,1]) 
              Mag_sus_xx[i][j] = float(L[j])*float(dtau)*np.dot(eq_sxsx_conn_r_nbr_avg,Adj_list[:,1])
              Mag_sus_xx_std[i][j] = float(L[j])*float(dtau)*np.sqrt(np.dot(np.power(eq_sxsx_conn_r_nbr_std_avg,2),Adj_list[:,1]))
              Mag_sus_zz[i][j] = float(L[j])*float(dtau)*np.dot(eq_szsz_conn_r_nbr_avg,Adj_list[:,1])
              Mag_sus_zz_std[i][j] = float(L[j])*float(dtau)*np.sqrt(np.dot(np.power(eq_szsz_conn_r_nbr_std_avg,2),Adj_list[:,1]))


              Spin_zz_correlation_connected[i,j,:] = np.copy(eq_szsz_conn_r_nbr_avg)
              Spin_zz_correlation_connected_var[i,j,:] = np.power(eq_szsz_conn_r_nbr_std_avg,2)                       
              Spin_zz_correlation_connected_tot[i,j] = np.dot(eq_szsz_conn_r_nbr_avg,Adj_list[:,1])
              Spin_zz_correlation_connected_tot_var[i,j] = np.dot(np.power(eq_szsz_conn_r_nbr_std_avg,2),Adj_list[:,1]) 
           
   for j in range(len(L)):
       
       data_mag_sus_xx = np.stack((mu_val[:,j],Density[:,j],Density_dev[:,j],Mag_sus_xx[:,j],Mag_sus_xx_std[:,j]),axis = 1)
       data_mag_sus_zz = np.stack((mu_val[:,j],Density[:,j],Density_dev[:,j],Mag_sus_zz[:,j],Mag_sus_zz_std[:,j]),axis = 1)
       
       np.savetxt("%s/Magnetic_susceptibility_xx_N_%s_U_%s_dtau_%s_L_%s.dat"%(Text_dir_mag_sus,N,u,dtau,L[j]),data_mag_sus_xx)
       np.savetxt("%s/Magnetic_susceptibility_zz_N_%s_U_%s_dtau_%s_L_%s.dat"%(Text_dir_mag_sus,N,u,dtau,L[j]),data_mag_sus_zz)
       

   for rr in range(len(r_2_unq)): 
       Pl_title_n1 = r"$\langle m^2_i m^2_j \rangle_{C}$ |i-j|=1.0, $N=%sx%s, U = %s$"%(N,N,u)

       Pl_y_lab = r"$\langle m^2_i m^2_j \rangle_{C}$"
       Pl_x_lab = r"$\langle n \rangle$"
       Pl_x_lab_mu = r"$\mu$"

    
       Pl_name_sxsx = "%s/Spin_xx_correlation_connected_%s_nearest_neighbor_normal_averaged_with_errorbars_N_%s_U_%s_dtau_%s.png"%(Graph_dir_spin_spin,str(rr),N,u,dtau)
       Pl_name_szsz = "%s/Spin_zz_correlation_connected_%s_nearest_neighbor_normal_averaged_with_errorbars_N_%s_U_%s_dtau_%s.png"%(Graph_dir_spin_spin,str(rr),N,u,dtau)
       plot_func_multiple_beta(N,u,Mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,Density,Spin_xx_correlation_connected[:,:,rr],Spin_xx_correlation_connected_var[:,:,rr],Pl_name_sxsx)
       plot_func_multiple_beta(N,u,Mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,Density,Spin_zz_correlation_connected[:,:,rr],Spin_zz_correlation_connected_var[:,:,rr],Pl_name_szsz)
   
   Pl_name_sxsx_tot = "%s/Spin_xx_correlation_connected_total_with_errorbars_N_%s_U_%s_dtau_%s.png"%(Graph_dir_spin_spin,N,u,dtau)
   Pl_name_szsz_tot = "%s/Spin_zz_correlation_connected_total_with_errorbars_N_%s_U_%s_dtau_%s.png"%(Graph_dir_spin_spin,N,u,dtau)
   plot_func_multiple_beta(N,u,Mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,Density,Spin_xx_correlation_connected_tot,Spin_xx_correlation_connected_tot_var,Pl_name_sxsx_tot)
   plot_func_multiple_beta(N,u,Mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,Density,Spin_zz_correlation_connected_tot,Spin_zz_correlation_connected_tot_var,Pl_name_szsz_tot)
     


In [115]:
def susceptibility_plot(Main_text_dir,Graph_dir_mag_sus,N,u,dtau,L):

    n_order = 5
    Text_dir_mag_sus = "%s/Text_files_N_%s_magnetic_susceptibility/Text_files_N_%s_U_%s"%(Main_text_dir,N,N,u)

    plt.figure(figsize = (30,20))
    plt.xticks(fontsize = 80)
    plt.yticks(fontsize = 80)
   
    color_1 = iter(cm.gnuplot(np.linspace(0, 1, len(L)+1)))
    
    for j in range(len(L)-1,-1,-1):
       c_1 = next(color_1) 
       filename_mag_sus_xx = "%s/Magnetic_susceptibility_xx_N_%s_U_%s_dtau_%s_L_%s.dat"%(Text_dir_mag_sus,N,u,dtau,L[j])
       m_val,nden,n_den_std,chi_xx,chi_xx_std = np.loadtxt(filename_mag_sus_xx, unpack = True, usecols = [0,1,2,3,4])    
       beta = (float(dtau))*(float(L[j]))
       T = round(1/beta,2)
       plt.errorbar(nden,chi_xx,yerr = chi_xx_std,c=c_1,marker = 'o',markersize = 15, linewidth = 5, elinewidth = 3, capsize = 5,label = r"$T = %s$"%str(T))
    plt.xlim(1,1.8)
    plt.ylim(0.1,1)
    plt.yscale('log')
    plt.grid()
    plt.axvline(x = 1,color="black", linestyle=":")
    plt.axhline(y = 0,color="black", linestyle=":")
    plt.legend(loc = 'best',ncol = 2,fontsize = 40)
    plt.savefig("%s/Magnetic_susceptibility_xx_N_%s_U_%s_dtau_%s.png"%(Graph_dir_mag_sus,N,u,dtau))


    plt.figure(figsize = (30,20))
    plt.xticks(fontsize = 80)
    plt.yticks(fontsize = 80)
   
    color_1 = iter(cm.gnuplot(np.linspace(0, 1, len(L)+1)))
    
    for j in range(len(L)-1,-1,-1):
       c_1 = next(color_1) 
       filename_mag_sus_zz = "%s/Magnetic_susceptibility_zz_N_%s_U_%s_dtau_%s_L_%s.dat"%(Text_dir_mag_sus,N,u,dtau,L[j])
       m_val,nden,n_den_std,chi_zz,chi_zz_std = np.loadtxt(filename_mag_sus_zz, unpack = True, usecols = [0,1,2,3,4])    
       beta = (float(dtau))*(float(L[j]))
       T = round(1/beta,2)
       plt.errorbar(nden,chi_zz,yerr = chi_zz_std,c=c_1,marker = 'o',markersize = 15, linewidth = 5, elinewidth = 3, capsize = 5,label = r"$T = %s$"%str(T))
    plt.xlim(1,1.8)
    plt.ylim(0.1,1)
    plt.yscale('log')
    plt.grid()
    plt.axvline(x = 1,color="black", linestyle=":")
    plt.axhline(y = 0,color="black", linestyle=":")
    plt.legend(loc = 'best',ncol = 2,fontsize = 40)
    plt.savefig("%s/Magnetic_susceptibility_zz_N_%s_U_%s_dtau_%s.png"%(Graph_dir_mag_sus,N,u,dtau))


    #========================================Plotting with running mean===============================================================

    plt.figure(figsize = (30,20))
    plt.xticks(fontsize = 80)
    plt.yticks(fontsize = 80)
   
    color_1 = iter(cm.gnuplot(np.linspace(0, 1, len(L)+1)))
    for j in range(len(L)-1,-1,-1):
       c_1 = next(color_1) 
       filename_mag_sus_xx = "%s/Magnetic_susceptibility_xx_N_%s_U_%s_dtau_%s_L_%s.dat"%(Text_dir_mag_sus,N,u,dtau,L[j])
       m_val,nden,n_den_std,chi_xx,chi_xx_std = np.loadtxt(filename_mag_sus_xx, unpack = True, usecols = [0,1,2,3,4])  
       n_rm = running_mean(nden,n_order)
       chi_xx_rm = running_mean(chi_xx,n_order)
       chi_xx_std_rm = np.sqrt(running_mean(np.power(chi_xx_std,2),n_order)) 
       beta = (float(dtau))*(float(L[j]))
       T = round(1/beta,2)
       plt.errorbar(n_rm,chi_xx_rm,yerr = chi_xx_std_rm,c=c_1,marker = 'o',markersize = 15, linewidth = 5, elinewidth = 3, capsize = 5,label = r"$T = %s$"%str(T))
    plt.xlim(1,1.8)
    plt.ylim(0.1,1)
    plt.yscale('log')
    plt.grid()
    plt.axvline(x = 1,color="black", linestyle=":")
    plt.axhline(y = 0,color="black", linestyle=":")
    plt.legend(loc = 'best',ncol = 2,fontsize = 40)
    plt.savefig("%s/Magnetic_susceptibility_xx_running_mean_order_%s_N_%s_U_%s_dtau_%s.png"%(Graph_dir_mag_sus,str(n_order),N,u,dtau))


    plt.figure(figsize = (30,20))
    plt.xticks(fontsize = 80)
    plt.yticks(fontsize = 80)
   
    color_1 = iter(cm.gnuplot(np.linspace(0, 1, len(L)+1)))
    for j in range(len(L)-1,-1,-1):
       c_1 = next(color_1) 
       filename_mag_sus_zz = "%s/Magnetic_susceptibility_zz_N_%s_U_%s_dtau_%s_L_%s.dat"%(Text_dir_mag_sus,N,u,dtau,L[j])
       m_val,nden,n_den_std,chi_zz,chi_zz_std = np.loadtxt(filename_mag_sus_zz, unpack = True, usecols = [0,1,2,3,4])  
       n_rm = running_mean(nden,n_order)
       chi_zz_rm = running_mean(chi_zz,n_order)
       chi_zz_std_rm = np.sqrt(running_mean(np.power(chi_zz_std,2),n_order)) 
       beta = (float(dtau))*(float(L[j]))
       T = round(1/beta,2)
       plt.errorbar(n_rm,chi_zz_rm,yerr = chi_zz_std_rm,c=c_1,marker = 'o',markersize = 15, linewidth = 5, elinewidth = 3, capsize = 5,label = r"$T = %s$"%str(T))
    plt.xlim(0.9,1.8)
    plt.ylim(0.1,1)
    plt.yscale('log')
    plt.grid()
    plt.axvline(x = 1,color="black", linestyle=":")
    plt.axhline(y = 0,color="black", linestyle=":")
    plt.legend(loc = 'best',ncol = 2,fontsize = 40)
    plt.savefig("%s/Magnetic_susceptibility_zz_running_mean_order_%s_N_%s_U_%s_dtau_%s.png"%(Graph_dir_mag_sus,str(n_order),N,u,dtau))

    #=================================Perform polynomial fitting to data=============================================

    Graph_dir_curve_fit = "%s/Polynomial_fits"%Graph_dir_mag_sus
    if not os.path.exists(Graph_dir_curve_fit):
       os.makedirs(Graph_dir_curve_fit)
        
    n_fit = np.arange(1.0,1.85, 0.05)
    Mag_sus_fit = np.zeros((len(n_fit),len(L)))
    T_val = np.zeros(len(L))

    for j in range(len(L)-1,-1,-1):
        
       T_val[j] = 1/(float(L[j])*float(dtau))
       filename_mag_sus_xx = "%s/Magnetic_susceptibility_xx_N_%s_U_%s_dtau_%s_L_%s.dat"%(Text_dir_mag_sus,N,u,dtau,L[j])
       m_val,nden,n_den_std,chi_xx,chi_xx_std = np.loadtxt(filename_mag_sus_xx, unpack = True, usecols = [0,1,2,3,4]) 
       n_rm = running_mean(nden,n_order)
       chi_xx_rm = running_mean(chi_xx,n_order)
       chi_xx_std_rm = np.sqrt(running_mean(np.power(chi_xx_std,2),n_order)) 
       beta = (float(dtau))*(float(L[j]))
       T = round(1/beta,2)
       z = np.polyfit(n_rm, chi_xx_rm, 3)
       f = np.poly1d(z)    
       Mag_sus_fit[:,j] = np.copy(f(n_fit))
       plt.figure(figsize = (30,20))
       plt.xticks(fontsize = 80)
       plt.yticks(fontsize = 80)
       plt.errorbar(n_rm,chi_xx_rm,yerr = chi_xx_std_rm, color = 'red',marker = 'o',markersize = 15, linestyle = 'none',linewidth = 5, elinewidth = 3, capsize = 5,label = r"$T = %s$"%str(T))
       plt.plot(n_fit,f(n_fit),linewidth = 2, label = "%s order fit"%str(n_order))
       plt.xlim(1,1.8)
       plt.ylim(0.1,1.5)
       plt.yscale('log')
       plt.grid()
       plt.axvline(x = 1,color="black", linestyle=":")
       plt.axhline(y = 0,color="black", linestyle=":")
       plt.legend(loc = 'best',ncol = 2,fontsize = 40)
       plt.savefig("%s/Magnetic_susceptibility_xx_running_mean_order_%s_N_%s_U_%s_dtau_%s_L_%s_fit.png"%(Graph_dir_curve_fit,str(n_order),N,u,dtau,L[j]))
       plt.close('all')

    color_2 = iter(cm.jet(np.linspace(0, 1, len(n_fit)+1)))
    plt.figure(figsize = (30,20))
    plt.xticks(fontsize = 80)
    plt.yticks(fontsize = 80)
    for i in range(len(n_fit)):
        c2=next(color_2)
        plt.plot(T_val,np.power(Mag_sus_fit[i,:],-1),c=c2,marker="o",markersize=15,linewidth = 5, label = "n=%s"%str(round(n_fit[i],3)))
    plt.xlabel("T",fontsize = 60)
    plt.ylabel(r"$\frac{1}{\chi}$",fontsize=80)
    plt.grid()
    plt.yscale('log')
    plt.xscale('log')

    plt.axvline(x = 1,color="black", linestyle=":")
    plt.axhline(y = 0,color="black", linestyle=":")
    plt.legend(loc = 'best',ncol = 2,fontsize = 40)
    plt.savefig("%s/Chi_xx_running_mean_polynomial_fit_order_%s_N_%s_U_%s_fit.png"%(Graph_dir_mag_sus,str(n_order),N,u))
    plt.close('all')


In [116]:
def main():

    N = "12"
    Dtau = "0.05"
    U = ["6.00","8.00"]
   # Mu = #["-15.00","-14.50","-14.00","-13.50","-13.00","-12.50","-12.00","-11.50","-11.00","-10.50","-10.00","-9.50","-9.00","-8.50",
         # "-8.00","-7.80","-7.60","-7.40","-7.20","-7.00","-6.80","-6.60","-6.40","-6.20","-6.00","-5.80","-5.60","-5.40","-5.20",
         # "-5.00","-4.80","-4.60","-4.40","-4.20"]
    Mu = ["-4.00","-3.80","-3.60","-3.40","-3.20","-3.00","-2.80","-2.60","-2.40","-2.20",
          "-2.00","-1.80","-1.60","-1.40","-1.20","-1.00","-0.80","-0.60","-0.40","-0.20","0.00","0.20","0.40","0.60","0.80","1.00","1.20","1.40","1.60","1.80","2.00","2.20","2.40","2.60","2.80","3.00","3.20","3.40","3.60","3.80",
          "4.00","4.20","4.40","4.60","4.80","5.00","5.20","5.40","5.60","5.80","6.00","6.20","6.40","6.60","6.80","7.00","7.20","7.40",
          "7.60","7.80","8.00","8.50","9.00","9.50","10.00","10.50","11.00","11.50","12.00","12.50","13.00","13.50","14.00","14.50","15.00"]
    
    Trot = ["10","12","14","16","18","20","22","24","26","28","30","32","34","36","38","40","42"]
    Interaction_val = np.zeros(len(U))
    Chemical_potential_val = np.zeros(len(Mu))

    Text_dir_main = os.getcwd()

    r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
    r_rel_x = r_grid[:,0]
    r_rel_y = r_grid[:,1]

    r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
    r_2_unq = np.sort(np.unique(r_2))

    
    for u_points in range(len(U)):
       
        Interaction_val[u_points] = float(U[u_points]) 
        Graph_dir_spin_spin = '../Graphs/Graphs_N_%s_U_%s_dtau_%s_real_space_correlations/Spin_correlations_neighbor_averaged_normal_average'%(N,U[u_points],Dtau)

        if not os.path.exists(Graph_dir_spin_spin):
            os.makedirs(Graph_dir_spin_spin)
         
        #equal_time_spin_spin_connected_correlation_function_real_space_plot(Text_dir_main,Graph_dir_spin_spin,N,U[u_points],Mu,Dtau,Trot)
        susceptibility_plot(Text_dir_main,Graph_dir_spin_spin,N,U[u_points],Dtau,Trot)




In [117]:
main()